<a href="https://colab.research.google.com/github/TurkuNLP/finngen-tools/blob/main/text_generation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation example

This is a brief example of how to run text generation with a Finnish GPT-3 model on Colab.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.3 MB/s eta 0:00:00


Import the `AutoTokenizer` and `AutoModelForCausalLM` classes. These support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models).

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [3]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Here's a simple function to generate text using broadly reasonable parameters. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

In [8]:
def generate(prompt, temperature=0.7, max_new_tokens=50):
    input = tokenizer(prompt, return_tensors='pt')
    output = model.generate(
        **input,
        do_sample=True,
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=2,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

A few generation examples. Note that re-running these generation examples will produce different outputs as `model.generate` is invoked with the `do_sample=True` parameter.

First, a simple sentence continuation:

In [14]:
print(generate('Suomen kieltä käsittelevien tekoälymenetelmien kehitykselle on keskeisen tärkeää, että'))

Suomen kieltä käsittelevien tekoälymenetelmien kehitykselle on keskeisen tärkeää, että myös suomen kielen ja sen lähikielen, ruotsin, tutkimusta tehdään Suomessa. Tämä ei ole mahdollista ilman vahvaa ruotsinkielistä resursointia. Ruotsinkielisen tutkimuksen tulee perustua tieteellisesti korkeatasoiseen tutkimukseen ja vahvaan yhteistyöhön muiden tutkimusyksiköiden kanssa.


An example of one-shot classification:

In [12]:
prompt = """Onko seuraava lause positiivinen vai negatiivinen?
Lause: Täällä on erittäin hauskaa!
Vastaus: positiivinen

Onko seuraava lause positiivinen vai negatiivinen?
Lause: elokuva oli surkea!
Vastaus:"""

print(generate(prompt))

Onko seuraava lause positiivinen vai negatiivinen?
Lause: Täällä on erittäin hauskaa!
Vastaus: positiivinen

Onko seuraava lause positiivinen vai negatiivinen?
Lause: elokuva oli surkea!
Vastaus: negatiivinen

Ovatko seuraavat lauseet positiivisia vai negatiivisia?
1. Mitä teet, kun olet stressaantunut?
2. Miltä sinusta tuntui tänään?
3. Mikä sinua ärsytti tänään?


Note above that even if the model produces the right answer, it usually doesn't know to stop generation after the answer and attempts to predict a plausible continuation.